In [ ]:
# comparison to best performing approach (final_version) is comparable with lower angles, and gets
# worse with higher angles 

# couldn't find any particular reasons that can explain the results


# possible problem with the angles:
# 1- thresholding can cause the angle to change

# differences between the two approaches
# 1- we use the set_upright and we save the angle to be passed later to the model (was expected to improve the 
#     performance rather than worsen it)
# 2- this method is trained on less samples, no data augmentation (though that shouldn't worsen the performance,
#     because set_upright should nulify the random rotation applied)

# so possibly the set_upright doesn't work as intended or saving the angle worsen the accuracy of the model

In [200]:
# random_angle_training = 0

# random_angle_test = 0
# base accuracy:  95.97

# random_angle_test = 15
# accuracy:  92.23

# random_angle_test = 30
# accuracy:  81.53

# random_angle_test = 45
# accuracy:  70.79

In [ ]:
# random_angle_training = 45

# random_angle_test = 0
# accuracy:  93.36

# random_angle_test = 15
# accuracy:  93.71

# random_angle_test = 30
# accuracy:  93.03

# random_angle_test = 45
# accuracy:  92.17

# random_angle_test = 60
# accuracy:  90.2

# random_angle_test = 75
# accuracy:  84.86

# random_angle_test = 90
# accuracy:  76.69

In [ ]:
# angle_training = 90

# random_angle_test = 0
# accuracy:  90.09

# random_angle_test = 15
# accuracy:  90.87

# random_angle_test = 30
# accuracy:  90.24

# random_angle_test = 45
# accuracy:  90.97

# random_angle_test = 60
# accuracy:  90.75

# random_angle_test = 75
# accuracy:  90.41

# random_angle_test = 90
# accuracy:  89.33

# random_angle_test = 105
# accuracy:  87.29

# random_angle_test = 120
# accuracy:  82.49

# random_angle_test = 135
# accuracy:  77.74

In [1]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from matplotlib.pyplot import imshow
import math
import os, os.path
import webbrowser
import imutils
import random

In [31]:
# here we are using contouring to find the angle of the image, and then using it to make the image upright

def new_random_rotate(path, angle_range):
    
    # dimenstions for resizing (width, height)
    dim = (28, 28)

    data = []
    labels = []
    valid_images = [".jpg"]

    for i in range(10):
        path = path + "\\" + str(i)
#     print(path)
        print(i)
        for f in os.listdir(path):
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_images:
                continue
            image = cv2.imread(os.path.join(path,f))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            
            random_angle = random.randint(-angle_range, angle_range)        
            rotated_image = imutils.rotate_bound(gray, angle= random_angle)
            ret,thresh = cv2.threshold(rotated_image,50,255,0)

            
            resized = cv2.resize(thresh, dim, interpolation = cv2.INTER_AREA)
#             flat_array = resized.flatten()
            data.append(resized)
            
            labels.append(i)
     
        path = path.rsplit('\\',1)[0]

    return (data, labels)

In [24]:
def new_set_upright(the_set):
    dim = (28, 28)
    data = []
    
    for gray_image in the_set:
#         ret,thresh = cv2.threshold(gray_image,10,255,0)
        rotation_angle = find_angle_image(gray_image)

        if rotation_angle > 45:
            rotated_image = imutils.rotate_bound(gray_image, angle= (90-rotation_angle))
        elif rotation_angle <= 45:
            rotated_image = imutils.rotate_bound(gray_image, angle= (rotation_angle))
            
        resized = cv2.resize(rotated_image, dim, interpolation = cv2.INTER_AREA)


        ret,thresh = cv2.threshold(resized,50,255,0)
        
        flat_array = thresh.flatten()
        
        data.append(flat_array)
    return data

In [25]:
def find_angle_image(image):
    
    contours, hierarchy = cv2.findContours(image, cv2.RETR_CCOMP , cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)
    rect = cv2.minAreaRect(cnt)
    rotation_angle = round(cv2.minAreaRect(cnt)[2])
    
    return rotation_angle

In [26]:
def add_angle_set(the_set):
    data=[]
    for image in the_set:
        rotation_angle= find_angle_image(image)
        flat_array = image.flatten()        
        flat_array = np.concatenate([[rotation_angle],flat_array])   
        data.append(flat_array)
        
    return data

In [40]:
#ADDED LATER

def calculate_accuracy(test_results, test_labels):
    error = 0
    for i in range(len(test_results)):
        if test_results[i] != test_labels[i]:
            error += 1
    return ((len(test_labels)-error)/len(test_labels)*100)

In [49]:
random_angle_training = 90


print("preprocessing: random rotate training data")
taining_path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_training"
(training_data, training_labels) = new_random_rotate(taining_path, random_angle_training)

preprocessing: random rotate training data
0
1
2
3
4
5
6
7
8
9


In [50]:
# print("preprocessing: normalize training data")
training_data = new_set_upright(training_data)

In [51]:
# print("finding the random angle: training set")
training_data=add_angle_set(training_data)

In [52]:
from sklearn.ensemble import RandomForestClassifier


# change max depth
# clf = RandomForestClassifier(max_depth=15, random_state=0)
# clf = RandomForestClassifier(max_depth=30, random_state=0)
clf = RandomForestClassifier(max_depth=50, random_state=0)


clf.fit(training_data, training_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [53]:
# added later
print("preprocessing: random rotate test data")
test_path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_testing"
results = []

for angle in range(0, 136, 15): 
    print("angle: ", angle)
    print("preprocessing: random rotate test data")
    test_path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_testing"
    (test_data, test_labels) = new_random_rotate(test_path, angle)
    
    # print("preprocessing: normalize test data")
    test_data = new_set_upright(test_data)
    
    # print("finding the random angle: test set")
    test_data =add_angle_set(test_data)
    
    test_results = clf.predict(test_data)
    results.append(calculate_accuracy(test_results, test_labels))

preprocessing: random rotate test data
angle:  0
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  15
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  30
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  45
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  60
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  75
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  90
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  105
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  120
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9
angle:  135
preprocessing: random rotate test data
0
1
2
3
4
5
6
7
8
9


In [54]:
print(results)

[90.09, 90.86999999999999, 90.24, 90.97, 90.75, 90.41, 89.33, 87.29, 82.49, 77.74]


In [ ]:
# random_angle_test = 180

# print("preprocessing: random rotate test data")
# test_path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_testing"
# (test_data, test_labels) = new_random_rotate(test_path, random_angle_test)

In [ ]:
# print("preprocessing: normalize test data")
# test_data = new_set_upright(test_data)

In [ ]:
# print("finding the random angle: test set")
# test_data =add_angle_set(test_data)

In [30]:
# test_results = clf.predict(test_data)

In [31]:
# print(test_results)

[0 0 0 ... 9 9 9]


In [32]:
# error = 0
# for i in range(len(test_results)):
#     if test_results[i] != test_labels[i]:
#         error += 1
# print("errors:", error)
# print("sample length: ", len(test_labels))
# print("accuracy: ", (len(test_labels)-error)/len(test_labels)*100)

errors: 1304
sample length:  10000
accuracy:  86.96000000000001
